In [ ]:
%pip install pandas numpy transformers matplotlib opencv-python

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu116

In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
!git clone https://huggingface.co/microsoft/trocr-large-handwritten

In [ ]:
#wget csak linuxon műkszik
!wget "https://huggingface.co/datasets/zhixiaoni/CROHME_selected_Train_2014_png/resolve/refs%2Fconvert%2Fparquet/default/train/0000.parquet"

import cv2
import matplotlib.pyplot as plt
i

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')
i_list = [0]

def OCR(row,i_list,processor,model):
    i_list[0]= i_list[0]+1
    print(i_list[0])
    nparr = np.fromstring(row["image"]["bytes"], np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    #plt.figure(figsize=(10,10)); plt.axis('off'); plt.imshow(img)
    #plt.show()
    pixel_values = processor(images=img, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    print(generated_text)
    return generated_text

In [ ]:
df = pd.read_parquet("./0000.parquet")
df

In [ ]:
df = df.head(3000)
df["text"] = df.apply(lambda x: OCR(x, i_list,processor,model), axis=1)

In [ ]:
df.shape[0]

In [ ]:
df.to_csv('ocr_results_3000.csv')

In [ ]:
df = pd.read_csv("ocr_results_3000.csv")

In [ ]:
def big_num_as_str(number):
    return "{:,}".format(number)

def get_patterns(ocr_results=['alma', 'korte', 'balma', 'almkortelma'], pattern_len=3):
    patterns = []
    for res in ocr_results:
        word_len = len(res)
        start_index = 0
        print(f"checking patterns with: {res}")
        for i in range(0, word_len-1):
            end_index = i+pattern_len
            possible_pattern = res[start_index : end_index]
            if (len(possible_pattern) != pattern_len):
                break
            print(f"\tpossible_patter: {possible_pattern}, indexes: {start_index}:{end_index}")

            #compare with other results
            for other_result in ocr_results:
                if (possible_pattern in other_result and other_result != res):
                    patterns.append(possible_pattern)
            start_index += 1
        print()
    print(f"results: {patterns}")
    return patterns

def filter_patterns(patterns, min_occurance=3):
    pattern_ocs = []
    values = {}
    temp_arr = []
    index = 0

    for pattern in patterns:
        print(f"index: {big_num_as_str(index)}, from : {big_num_as_str(len(patterns))}")
        index += 1
        if (pattern not in temp_arr):
            temp_arr.append(pattern)
            counter = -1
            for other in patterns:
                if (other == pattern):
                    counter += 1
            if (counter >= min_occurance):
                print(f'new pattern found: {pattern} - occurences: {counter}')
                values = {}
                values["pattern"] = pattern
                values["counter"] = counter
                pattern_ocs.append(values)
    return pattern_ocs

In [ ]:
possible_patterns_3 = get_patterns(ocr_results=df['text'].to_list(), pattern_len=3)
possible_patterns_4 = get_patterns(ocr_results=df['text'].to_list(), pattern_len=4)
possible_patterns_5 = get_patterns(ocr_results=df['text'].to_list(), pattern_len=5)

In [ ]:
possible_patterns_3.extend(possible_patterns_4)
possible_patterns_3.extend(possible_patterns_5)
possible_patterns = possible_patterns_3

In [ ]:
patterns = filter_patterns(patterns=possible_patterns, min_occurance=5)
#print(patterns)
pattern_df = pd.DataFrame.from_records(patterns)
pattern_df = pattern_df.drop_duplicates()
pattern_df.to_csv("patterns.csv", index=False)

pattern_df

In [ ]:
pattern_df = pd.read_csv("patterns.csv")
pattern_df = pattern_df.sort_values(by=["counter"], ascending=False)
pattern_df

In [ ]:
# PATTERNEK-BŐL KI KELLENE SZŰRNI AZ ANGOL SZAVAKAT
import json

en_dict = json.load(open("words_dictionary.json"))
wrong_indexes = []

for index, row in pattern_df.iterrows():
    el = row['pattern']
    if el in en_dict:
        wrong_indexes.append(index)
        #print(f'found this: {el} on index {index}')
    elif el.replace(" ", "") in en_dict:
        wrong_indexes.append(index)

print(pattern_df.count)
pattern_df.drop(wrong_indexes,axis=0,inplace=True)
print(pattern_df.count)

pattern_df.to_csv("patterns_no_word.csv", index=False)

In [ ]:
#img = cv2.imread("test_imgs/mixed_test_2.png") # fail probably text
#img = cv2.imread("test_imgs/mixed_test_3.png") # fail probably text
#img = cv2.imread("test_imgs/mixed_test_4.png") # fail math
#img = cv2.imread("test_imgs/mixed_test_5.png") # fail math
#img = cv2.imread("test_imgs/mixed_test_6.png") # fail probably text
#img = cv2.imread("test_imgs/math_test_8.png") # fail probably text
#img = cv2.imread("test_imgs/math_test_13.png") # fail probably text
# img = cv2.imread("test_imgs/text_test_7.png")  #fail probably text
#img = cv2.imread("test_imgs/math_test_16.png") #fail probably text
#img = cv2.imread("test_imgs/math_test_18.png") #fail probably text
#img = cv2.imread("test_imgs/math_test_19.png") #fail probably text
#img = cv2.imread("test_imgs/math_test_19.png")

#img = cv2.imread("test_imgs/math_test_4.png") #ok
#img = cv2.imread("test_imgs/math_test_5.png") #ok
#img = cv2.imread("test_imgs/math_test_6.png") # ok
#img = cv2.imread("test_imgs/math_test_7.png") #ok
#img = cv2.imread("test_imgs/math_test_8.png") #elég edge case, ezt adja az ocr: 'Sebards #'

img = cv2.imread("test_imgs/text_test_8.png")
#img = cv2.imread("test_imgs/text_test_easy.png")

In [ ]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel

processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')

In [ ]:
pixel_values = processor(images=img, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

In [ ]:
def check_distances(found_patterns, text, max_char_distance):
    #megnézi, h van-e legalább egy olyan pattern ami elég közel van hozzá
    num_of_close = 0
    close_pairs = []
    for i in range(len(found_patterns)):
        for j in range(len(found_patterns)):
            if j > i:
                char_distance = abs(text.find(found_patterns[i]) - text.find(found_patterns[j]))
                if max_char_distance >= char_distance and char_distance > 1:
                    close_pairs.append([found_patterns[i], found_patterns[j]])
                    num_of_close = num_of_close+1
                    break
    '''for pair in close_pairs:
        print(pair)'''
    return num_of_close

def check_patterns_in_string(text, cutoff =.45 ,max_char_distance=5, do_prints=True, csv_to_read_from='patterns_no_word.csv'):
    pattern_df = pd.read_csv(csv_to_read_from)
    patterns_as_list = pattern_df['pattern'].to_list()
    if do_prints == True: print(f"Checking patterns in '{text}'")
    
    found_patterns = []
    pattern_len = len(patterns_as_list[0])
    text_len = len(text)
    possible_pattern_count = text_len-pattern_len+1
    if do_prints == True: print(f"text_len: {text_len}, possible_pattern_count: {possible_pattern_count}")    

    #if do_prints == True: print(patterns_as_list)
    for pattern in patterns_as_list:
        if (pattern in text):
            found_patterns.append(pattern)

    if do_prints == True: print(f"\nfound {len(found_patterns)} patterns in the text")
    if do_prints == True: print(found_patterns)
    
    if (round(possible_pattern_count * cutoff) <= len(found_patterns)):
        # check distance of words
        min_num_of_close = len(found_patterns) * .75
        num_of_close = check_distances(found_patterns, text, max_char_distance)

        print(f'num_of_close: {num_of_close}, min: {len(found_patterns)}*.75={min_num_of_close}')
        if num_of_close >= min_num_of_close:
            if do_prints == True: print("math")
            return 'math'            
        else:
            if do_prints == True: print("mixed")
            return 'mixed'

    elif (round(possible_pattern_count * cutoff) > len(found_patterns)):
        if do_prints == True: print('probably text')
        return 'probably text'
    else:
        if do_prints == True: print('pretty sure its text')
        return 'pretty sure its text'

In [ ]:
check_patterns_in_string(generated_text, cutoff=.45, do_prints=True)

In [ ]:
import os

def test_pattern_check(img_type='math_test', correct_results=['math'], csv_to_read_patterns_from='patterns_no_word.csv', print_inner_logs=True):
    counter = 1
    tests_good = []
    tests_bad = []
    num_of_imgs = 0
    img_paths = []
    for subdir, dirs, files in os.walk('test_imgs'):
        for file in files:
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + file
            if img_type in filepath: 
                num_of_imgs += 1
                img_paths.append(filepath)
    print(img_paths)
    print(num_of_imgs)
    for img_path in img_paths:
        img = cv2.imread(img_path)
        #ocr
        pixel_values = processor(images=img, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values)
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(f'generated this string: "{generated_text}" from img {img_path}')
        #checking patterns
        result = check_patterns_in_string(generated_text, cutoff=.45, do_prints=print_inner_logs, csv_to_read_from=csv_to_read_patterns_from)
        print(result)
        if result in correct_results:
            tests_good.append(True)
        else:
            tests_bad.append(False)
        print(f'processed: {counter}/{num_of_imgs}\n')
        counter += 1
    print(f'Successrate = {(len(tests_good)/(len(tests_good)+len(tests_bad)))*100}%')

In [ ]:
test_pattern_check(img_type='math_test', correct_results=['math'])

In [ ]:
test_pattern_check(img_type='mixed_test', correct_results=['mixed'])

In [ ]:
test_pattern_check(img_type='text_test', correct_results=['pretty sure its text', 'probably text'])